In [86]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, ARIMA
import sys


df = pd.read_excel("planilhas/dados.xlsx")
total_linhas = len(df)


In [87]:

display(df)

# Função para remover caracteres não numéricos
def convert(coluna):
    return pd.to_numeric(coluna, errors='coerce')


# Usar a função para converter a coluna específica
df['saeb_Mat_2019'] = convert(df['saeb_Mat_2019'])
df['saeb_Port_2019'] = convert(df['saeb_Port_2019'])

df['saeb_Mat_2023'] = convert(df['saeb_Mat_2023'])
df['saeb_Port_2023'] = convert(df['saeb_Port_2023'])

display(df)


,uf,codigo,municipio,cod.escola,escola,rede,TXApro_Total_2017,TXApro_1aSERIE_2017,TXApro_2aSERIE_2017,TXApro_3aSERIE_2017,...,saeb_Media_padronizada_2021,saeb_Mat_2023,saeb_Port_2023,saeb_Media_padronizada_2023,IDEB_2017,IDEB_2019,IDEB_2021,IDEB_2023,Meta_2019,Meta_2021
0,RO,1100015,Alta Floresta D'Oeste,11024682,EEEFM EURIDICE LOPES PEDROSO,Estadual,NaN,NaN,NaN,NaN,...,3.899135,254.41,253.38,4.055802,NaN,NaN,3.6,4.0,NaN,NaN
1,RO,1100015,Alta Floresta D'Oeste,11024968,EEEMTI JUSCELINO KUBITSCHEK DE OLIVEIRA,Estadual,96.1,96.2,95.1,96.4,...,5.409930,306.56,300.31,5.490791,5.0,5.5,5.4,5.4,5.2,5.4
2,RO,1100015,Alta Floresta D'Oeste,11025638,COLEGIO TIRADENTES DA POLICIA MILITAR - CTPM XI,Estadual,85.1,80.2,89.0,87.8,...,NaN,285.14,288.04,5.006266,3.4,4.1,NaN,4.9,3.6,3.8
3,RO,1100023,Ariquemes,11006773,EEEFM CORA CORALINA,Estadual,NaN,NaN,NaN,NaN,...,4.469767,256.07,254.76,4.099775,NaN,NaN,2.8,2.4,NaN,NaN
4,RO,1100023,Ariquemes,11006889,EEEFM ANISIO TEIXEIRA,Estadual,70.2,62.8,68.8,86.3,...,NaN,264.11,261.11,4.307757,3.1,NaN,NaN,3.8,3.3,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21573,DF,5300108,Brasília,53056000,COL OLIMPO,Privada,98.2,96.2,98.3,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21574,DF,5300108,Brasília,53068068,CED DONA AMERICA GUIMARAES,Estadual,77.8,77.1,73.5,87.1,...,NaN,246.86,237.18,3.707248,3.4,3.8,NaN,3.4,3.6,3.8
21575,DF,5300108,Brasília,53068092,CED 01 DO RIACHO FUNDO II,Estadual,65.7,50.0,70.6,87.7,...,4.034805,NaN,NaN,NaN,NaN,4.0,3.1,NaN,NaN,4.2
21576,DF,5300108,Brasília,53068181,CED 310 DE SANTA MARIA,Estadual,76.8,69.5,84.6,79.0,...,NaN,260.74,258.48,4.221054,NaN,NaN,NaN,3.2,NaN,NaN


,uf,codigo,municipio,cod.escola,escola,rede,TXApro_Total_2017,TXApro_1aSERIE_2017,TXApro_2aSERIE_2017,TXApro_3aSERIE_2017,...,saeb_Media_padronizada_2021,saeb_Mat_2023,saeb_Port_2023,saeb_Media_padronizada_2023,IDEB_2017,IDEB_2019,IDEB_2021,IDEB_2023,Meta_2019,Meta_2021
0,RO,1100015,Alta Floresta D'Oeste,11024682,EEEFM EURIDICE LOPES PEDROSO,Estadual,NaN,NaN,NaN,NaN,...,3.899135,254.41,253.38,4.055802,NaN,NaN,3.6,4.0,NaN,NaN
1,RO,1100015,Alta Floresta D'Oeste,11024968,EEEMTI JUSCELINO KUBITSCHEK DE OLIVEIRA,Estadual,96.1,96.2,95.1,96.4,...,5.409930,306.56,300.31,5.490791,5.0,5.5,5.4,5.4,5.2,5.4
2,RO,1100015,Alta Floresta D'Oeste,11025638,COLEGIO TIRADENTES DA POLICIA MILITAR - CTPM XI,Estadual,85.1,80.2,89.0,87.8,...,NaN,285.14,288.04,5.006266,3.4,4.1,NaN,4.9,3.6,3.8
3,RO,1100023,Ariquemes,11006773,EEEFM CORA CORALINA,Estadual,NaN,NaN,NaN,NaN,...,4.469767,256.07,254.76,4.099775,NaN,NaN,2.8,2.4,NaN,NaN
4,RO,1100023,Ariquemes,11006889,EEEFM ANISIO TEIXEIRA,Estadual,70.2,62.8,68.8,86.3,...,NaN,264.11,261.11,4.307757,3.1,NaN,NaN,3.8,3.3,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21573,DF,5300108,Brasília,53056000,COL OLIMPO,Privada,98.2,96.2,98.3,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21574,DF,5300108,Brasília,53068068,CED DONA AMERICA GUIMARAES,Estadual,77.8,77.1,73.5,87.1,...,NaN,246.86,237.18,3.707248,3.4,3.8,NaN,3.4,3.6,3.8
21575,DF,5300108,Brasília,53068092,CED 01 DO RIACHO FUNDO II,Estadual,65.7,50.0,70.6,87.7,...,4.034805,NaN,NaN,NaN,NaN,4.0,3.1,NaN,NaN,4.2
21576,DF,5300108,Brasília,53068181,CED 310 DE SANTA MARIA,Estadual,76.8,69.5,84.6,79.0,...,NaN,260.74,258.48,4.221054,NaN,NaN,NaN,3.2,NaN,NaN


In [88]:
with open('colunas.txt', 'w', encoding='utf-8') as f:
    sys.stdout = f
    df.info()

    f.write("\n\n")
    sys.stdout = sys.__stdout__

    for i, coluna in enumerate(df.columns):
        non_null_count = df[coluna].notnull().sum()  
        null_percentage = (non_null_count / total_linhas) * 100  
        dtype = str(df[coluna].dtype) 
        
        f.write(f"{i: <4} {coluna: <33} {non_null_count: <16} {dtype: <10}\n")
        
        f.write(f"     -> {null_percentage:.2f}% de dados não nulos.\n")

    # Escrever o total de linhas
    f.write(f"\nTotal de linhas no DataFrame: {total_linhas}\n")

In [89]:
dfEstados = (df.groupby('uf').mean(numeric_only=True).reset_index())

display(dfEstados)
dfEstados = dfEstados.drop(columns=['codigo', 'cod.escola'])

display(dfEstados)

dfEstados.to_csv("planilhas/estadosMedia.csv", index=False, sep=";", encoding="UTF-8", float_format='%.3f')



,uf,codigo,cod.escola,TXApro_Total_2017,TXApro_1aSERIE_2017,TXApro_2aSERIE_2017,TXApro_3aSERIE_2017,TXApro_4aSERIE_2017,Indicador de Rendimento (P)_2017,TXApro_Total_2019,...,saeb_Media_padronizada_2021,saeb_Mat_2023,saeb_Port_2023,saeb_Media_padronizada_2023,IDEB_2017,IDEB_2019,IDEB_2021,IDEB_2023,Meta_2019,Meta_2021
0,AC,1.200373e+06,1.201886e+07,87.491111,82.217045,88.809091,93.383333,95.750000,0.878149,85.212644,...,4.739328,263.566393,266.257705,4.377183,3.985106,3.943902,4.572727,3.954098,4.204255,4.326154
1,AL,2.704764e+06,2.705677e+07,85.790909,80.807950,87.305000,91.769295,89.610000,0.862170,87.848052,...,4.104776,258.430268,259.377589,4.202051,3.661875,3.763547,3.823881,3.987500,3.872500,4.041350
2,AM,1.302394e+06,1.309181e+07,85.349624,81.674300,84.934010,90.172180,NaN,0.851662,84.053824,...,3.967176,252.807391,255.730725,4.068484,3.585484,3.592308,3.772727,3.711232,3.789785,3.929237
3,AP,1.600380e+06,1.600500e+07,76.462617,69.628302,79.253774,84.307477,91.580000,0.765075,77.491860,...,NaN,250.411905,258.657381,4.078652,3.589286,3.265625,NaN,3.661905,3.796429,3.728302
4,BA,2.918755e+06,2.926302e+07,74.633771,65.842619,77.621353,84.857793,80.130000,0.745389,78.888931,...,4.378789,252.923665,256.474551,4.081252,3.278391,3.477244,4.030137,3.771956,3.506667,3.674711
5,CE,2.306422e+06,2.316367e+07,89.367540,85.477875,89.774891,94.541776,100.000000,0.896036,92.851908,...,4.658804,274.245617,276.732868,4.683986,4.073171,4.428076,4.578694,4.565825,4.280793,4.498540
6,DF,5.300108e+06,5.301119e+07,83.549618,77.598473,86.490840,90.042748,NaN,0.839019,82.801010,...,4.758932,270.115800,275.822400,4.612353,5.141176,4.175556,4.260870,4.112000,5.335294,4.904110
7,ES,3.203088e+06,3.204274e+07,87.476991,80.763664,89.119288,95.829794,94.088235,0.878218,90.079787,...,5.072980,282.890221,283.565147,4.907679,4.565678,4.684649,4.808434,4.740809,4.774576,4.921886
8,GO,5.210425e+06,5.204863e+07,92.998667,90.497064,92.705185,96.100000,90.150000,0.929992,93.183844,...,4.620805,279.803650,282.744228,4.852039,4.367925,4.659420,4.510708,4.787601,4.577736,4.776176
9,MA,2.107003e+06,2.118559e+07,86.603295,82.760153,87.437405,91.120786,NaN,0.866592,90.800132,...,3.883918,248.865626,251.401333,3.948311,3.457028,3.559720,3.591943,3.677561,3.669277,3.788919


,uf,TXApro_Total_2017,TXApro_1aSERIE_2017,TXApro_2aSERIE_2017,TXApro_3aSERIE_2017,TXApro_4aSERIE_2017,Indicador de Rendimento (P)_2017,TXApro_Total_2019,TXApro_1aSERIE_2019,TXApro_2aSERIE_2019,...,saeb_Media_padronizada_2021,saeb_Mat_2023,saeb_Port_2023,saeb_Media_padronizada_2023,IDEB_2017,IDEB_2019,IDEB_2021,IDEB_2023,Meta_2019,Meta_2021
0,AC,87.491111,82.217045,88.809091,93.383333,95.750000,0.878149,85.212644,80.095294,86.343529,...,4.739328,263.566393,266.257705,4.377183,3.985106,3.943902,4.572727,3.954098,4.204255,4.326154
1,AL,85.790909,80.807950,87.305000,91.769295,89.610000,0.862170,87.848052,83.153275,88.664935,...,4.104776,258.430268,259.377589,4.202051,3.661875,3.763547,3.823881,3.987500,3.872500,4.041350
2,AM,85.349624,81.674300,84.934010,90.172180,NaN,0.851662,84.053824,81.189714,83.501709,...,3.967176,252.807391,255.730725,4.068484,3.585484,3.592308,3.772727,3.711232,3.789785,3.929237
3,AP,76.462617,69.628302,79.253774,84.307477,91.580000,0.765075,77.491860,71.414118,79.009412,...,NaN,250.411905,258.657381,4.078652,3.589286,3.265625,NaN,3.661905,3.796429,3.728302
4,BA,74.633771,65.842619,77.621353,84.857793,80.130000,0.745389,78.888931,71.333495,81.260766,...,4.378789,252.923665,256.474551,4.081252,3.278391,3.477244,4.030137,3.771956,3.506667,3.674711
5,CE,89.367540,85.477875,89.774891,94.541776,100.000000,0.896036,92.851908,89.722137,92.988702,...,4.658804,274.245617,276.732868,4.683986,4.073171,4.428076,4.578694,4.565825,4.280793,4.498540
6,DF,83.549618,77.598473,86.490840,90.042748,NaN,0.839019,82.801010,76.534343,85.394949,...,4.758932,270.115800,275.822400,4.612353,5.141176,4.175556,4.260870,4.112000,5.335294,4.904110
7,ES,87.476991,80.763664,89.119288,95.829794,94.088235,0.878218,90.079787,85.616429,91.244444,...,5.072980,282.890221,283.565147,4.907679,4.565678,4.684649,4.808434,4.740809,4.774576,4.921886
8,GO,92.998667,90.497064,92.705185,96.100000,90.150000,0.929992,93.183844,90.372184,93.942662,...,4.620805,279.803650,282.744228,4.852039,4.367925,4.659420,4.510708,4.787601,4.577736,4.776176
9,MA,86.603295,82.760153,87.437405,91.120786,NaN,0.866592,90.800132,89.078273,90.483155,...,3.883918,248.865626,251.401333,3.948311,3.457028,3.559720,3.591943,3.677561,3.669277,3.788919


In [90]:
## TESTAR NUMERO

coluna = 'TXApro_3aSERIE_2017'
estado = 'RR'
dfTeste = df.loc[(df['uf']) == estado]
dfTeste[coluna].mean()


91.84347826086956

# CIDADES PARANÁ


In [91]:
dfCidades = df.loc[(df['uf']) == "PR"]

dfCidades = (df.groupby('municipio').mean(numeric_only=True).reset_index())

dfCidades = dfCidades.drop(columns=['codigo', 'cod.escola'])
display(dfCidades)


dfCidades.to_csv("planilhas/cidadesPR.csv", index=False, sep=";", encoding="UTF-8", float_format='%.3f')

,municipio,TXApro_Total_2017,TXApro_1aSERIE_2017,TXApro_2aSERIE_2017,TXApro_3aSERIE_2017,TXApro_4aSERIE_2017,Indicador de Rendimento (P)_2017,TXApro_Total_2019,TXApro_1aSERIE_2019,TXApro_2aSERIE_2019,...,saeb_Media_padronizada_2021,saeb_Mat_2023,saeb_Port_2023,saeb_Media_padronizada_2023,IDEB_2017,IDEB_2019,IDEB_2021,IDEB_2023,Meta_2019,Meta_2021
0,Abadia de Goiás,93.40000,93.50000,94.40,91.70,NaN,0.931864,97.800,97.600000,95.700,...,NaN,273.320000,285.580000,4.803428,4.00,4.600000,NaN,4.800000,4.20,4.400000
1,Abadia dos Dourados,89.50000,71.60000,95.40,100.00,NaN,0.870861,90.100,82.100000,95.500,...,4.840216,266.130000,286.530000,4.716666,4.40,NaN,4.7,4.400000,4.60,4.800000
2,Abadiânia,95.15000,96.50000,98.00,91.15,NaN,0.951009,93.900,94.300000,95.500,...,4.199450,272.330000,275.340000,4.636230,4.70,NaN,4.2,4.600000,4.90,5.100000
3,Abaetetuba,77.36875,71.44375,86.34,87.14,NaN,0.825605,84.465,79.978947,84.765,...,NaN,248.755263,247.424737,3.887231,3.08,3.263636,NaN,3.836842,3.28,3.358333
4,Abaeté,72.90000,61.60000,73.80,87.00,94.4,0.771138,71.800,62.000000,74.000,...,NaN,286.410000,289.860000,5.051348,4.30,3.900000,NaN,4.000000,4.50,4.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5283,Áurea,99.10000,100.00000,100.00,97.70,NaN,0.992214,98.100,100.000000,97.300,...,NaN,286.760000,290.830000,5.070785,NaN,4.500000,NaN,4.900000,NaN,4.700000
5284,Ângulo,86.20000,86.50000,78.80,95.80,NaN,0.864816,94.100,96.600000,92.600,...,4.453839,NaN,NaN,NaN,NaN,4.300000,4.2,NaN,NaN,4.600000
5285,Érico Cardoso,85.30000,82.60000,84.30,89.80,NaN,0.854584,90.400,89.300000,91.800,...,3.951500,267.300000,269.190000,4.473518,3.10,3.400000,3.7,4.300000,3.30,3.600000
5286,Óbidos,68.75000,57.05000,66.10,89.65,NaN,0.683426,71.800,59.350000,74.450,...,NaN,257.780000,262.547500,4.240372,2.30,2.650000,NaN,4.150000,2.60,2.900000
